# Tutorial: Train your first ML model (Part 2 of 3)

## Introduction

In the [previous tutorial](1.hello-world.ipynb), you ran a trivial "hello world!" script in the cloud using Azure Machine Learning's Python SDK. This time you take it a step further by submitting a script that will train a machine learning model. This example will help you understand how Azure Machine Learning eases consistent behavior between debugging on a compute instance or laptop development environment, and remote runs.

Learning these concepts means that by the end of this session, you can:

* Use Conda to define an Azure Machine Learning environment
* Train a model in the cloud
* Log metrics to Azure Machine Learning

---

## Your machine learning code

This tutorial shows you how to train a PyTorch model on the CIFAR 10 dataset using an Azure Machine Learning Cluster. In this case you will be using a CPU cluster, but this could equally be a GPU cluster. Whilst this tutorial uses PyTorch, the steps we show you apply to *any* machine learning code. 

Note the code is based on [this introductory example from PyTorch](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html). 

## Submit your machine learning code to Azure Machine Learning

The difference to the control script below and the one used to submit "hello world" is that you adjust the environment to be set from the conda dependencies file you created earlier.

> <span style="color:purple; font-weight:bold">! NOTE <br>
> The first time you run this script, Azure Machine Learning will build a new docker image from your PyTorch environment. The whole run could take 5-10 minutes to complete. You can see the docker build logs in the widget by selecting the `20_image_build_log.txt` in the log files dropdown. This image will be reused in future runs making them run much quicker.</span>


In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.widgets import RunDetails

ws = Workspace.from_config()
env = Environment.from_conda_specification(
    name="pytorch-tutorial",
    file_path="environment.yml",
)
exp = Experiment(workspace=ws, name="an-introduction-train-tutorial")
src = ScriptRunConfig(
    source_directory="src",
    script="train.py",
    compute_target="cpu-cluster",
    environment=env,
)

run = exp.submit(src)
RunDetails(run).show()

### Understand the control code

Compared to the control script that submitted the "hello world" example, this control script introduces the following:

| Code | Description
| --- | --- |
| `env = Environment.from_conda_specification( ...)` | Azure Machine Learning provides the concept of an `Environment` to represent a reproducible, <br>versioned Python environment for running experiments. Here you have created it from a yaml conda dependencies file.|

**There are many ways to create AML environments, including [from a pip requirements.txt](https://docs.microsoft.com/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py&preserve-view=true#from-pip-requirements-name--file-path-), or even [from an existing local Conda environment](https://docs.microsoft.com/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py&preserve-view=true#from-existing-conda-environment-name--conda-environment-name-).**


Once your image is built, select `70_driver_log.txt` to see the output of your training script, which should look like:

```txt
Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz
...
Files already downloaded and verified
epoch=1, batch= 2000: loss 2.19
...
epoch=2, batch=12000: loss 1.27
Finished Training
```

Environments can be registered to a workspace with `env.register(ws)`, allowing them to be easily shared, reused, and versioned. Environments make it easy to reproduce previous results and to collaborate with your team.

Azure Machine Learning also maintains a collection of curated environments. These environments cover common ML scenarios and are backed by cached Docker images. Cached Docker images make the first remote run faster.

In short, using registered environments can save you time! More details can be found on the [environments documentation](./how-to-use-environments.md).

In [ ]:
run.wait_for_completion(show_output=True)

## Log training metrics

Now that you have a model training in Azure Machine Learning, start tracking some performance metrics.
The current training script prints metrics to the terminal. Azure Machine Learning provides a
mechanism for logging metrics with more functionality. By adding a few lines of code, you gain the ability to visualize metrics in the studio and to compare metrics between multiple runs.

### Machine learning code updates

In the `src` directory you will notice the [train-with-logging.py](src/train-with-logging.py) script has been modified with one additional line that will log the loss to the Azure Machine Learning Studio:

```python
# in train.py
import mlflow
...
mlflow.log_metric("loss", loss)
```

Metrics in Azure Machine Learning are:

- Organized by experiment and run so it's easy to keep track of and
compare metrics
- Equipped with a UI so we can visualize training performance in the studio or in the notebook widget
- **Designed to scale** You can submit concurrent experiments and the Azure Machine Learning cluster will scale out (up to the maximum node count of the cluster) to run the experiments in parallel

### Submit your machine learning code to Azure Machine Learning
Submit your code once more. This time the widget includes the metrics where you can now see live updates on the model training loss!

In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.widgets import RunDetails

ws = Workspace.from_config()
env = Environment.from_conda_specification(
    name="pytorch-env-tutorial",
    file_path="environment.yml",
)
exp = Experiment(workspace=ws, name="an-introduction-train-with-logging-tutorial")
src = ScriptRunConfig(
    source_directory="src",
    script="train-with-logging.py",
    compute_target="cpu-cluster",
    environment=env,
)

run = exp.submit(src)
RunDetails(run).show()

## Next steps

In this session, you upgraded from a basic "hello world!" script to a more realistic
training script that required a specific Python environment to run. You saw how
to take a local Conda environment to the cloud with Azure Machine Learning Environments. Finally, you
saw how in a few lines of code you can log metrics to Azure Machine Learning.

In the next session, you'll see how to work with data in Azure Machine Learning by uploading the CIFAR10
dataset to Azure.

[Tutorial: Bring your own data](3.pytorch-model-cloud-data.ipynb)


In [ ]:
run.wait_for_completion(show_output=True)